In [4]:
import os
import re
import pandas as pd

# 1) Helper function to process one receitas file:
def load_and_label_receitas(filepath: str) -> pd.DataFrame:
    """
    Reads a single 'receitas-cobradas-liquidas-YYYY.xlsx' file:
      - Grabs header labels from Excel row 12
      - Reads data from Excel row 18 onward
      - Selects the specific merged & unmerged columns you specified
      - Names them ('autarquia', 'nif', 'date', + labels from row 12)
      - Cleans up any '.0' suffixes on header names
      - Drops any duplicate column names
      - Appends a 'year' column extracted from the filename
    """
    # --- 1a) Read header row 12 for labels of column 4 and beyond ---
    headers_df = pd.read_excel(filepath, header=None, nrows=12)
    header_labels = headers_df.iloc[11]   # zero‑based → Excel row 12

    # --- 1b) Read data from Excel row 18 onward ---
    data_df = pd.read_excel(filepath, header=None, skiprows=17)

    # --- 1c) Select only the columns per your new rules ---
    n_cols = data_df.shape[1]
    selected_indices = [2, 3, 5, 9] + list(range(11, n_cols))
    df = data_df.iloc[:, selected_indices].copy()

    # --- 1d) Build the column names ---
    col_names = ['autarquia', 'nif', 'date']
    for idx in selected_indices[3:]:
        col_names.append(str(header_labels[idx]))
    df.columns = col_names

    # --- 1e) Clean up any trailing '.0' on header names (caused by numeric floats) ---
    cleaned = []
    for col in df.columns:
        if isinstance(col, str) and col.endswith('.0'):
            cleaned.append(col[:-2])
        else:
            cleaned.append(col)
    df.columns = cleaned

    # --- 1f) Drop any duplicate column names (keep first occurrence) ---
    df = df.loc[:, ~df.columns.duplicated()]

    # --- 1g) Extract the year from the filename and append it ---
    fname = os.path.basename(filepath)
    m = re.search(r'receitas-cobradas-liquidas-(\d{4})\.xls[x]?$', fname, re.IGNORECASE)
    df['year'] = m.group(1) if m else None

    return df

# 2) Directory + filename pattern for receitas files
input_folder = './input_datasets/financial'  # update to your folder path
pattern = re.compile(r'receitas-cobradas-liquidas-\d{4}\.xls[x]?$', re.IGNORECASE)

# 3) Loop through matching files, load and collect
all_dfs = []
for fn in os.listdir(input_folder):
    if pattern.match(fn):
        full_path = os.path.join(input_folder, fn)
        all_dfs.append(load_and_label_receitas(full_path))

# 4) Concatenate into one DataFrame
receitas_df = pd.concat(all_dfs, ignore_index=True)

# 5) Drop any columns whose name is NaN (in case header_labels had gaps)
receitas_df = receitas_df.loc[:, receitas_df.columns.notna()]

# 6) Remove leading apostrophes from all column names
receitas_df.columns = [
    col.lstrip("'") if isinstance(col, str) else col
    for col in receitas_df.columns
]

# 7) Export to Excel
output_folder = './datasets/financial'
output_filename = 'receitas_combinadas.xlsx'
output_path = os.path.join(output_folder, output_filename)
os.makedirs(output_folder, exist_ok=True)

receitas_df.to_excel(
    output_path,
    index=False,
    engine='openpyxl'
)

print(f"Exported receitas_df to Excel at: {output_path}")


Exported receitas_df to Excel at: ./datasets/financial/receitas_combinadas.xlsx
